In [56]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim  as optim
import torch.utils.data

In [57]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


In [58]:
Y_train = []
for y in y_train_valid:
    value = np.abs(769-y)
    Y_train.append(value)
Y_train = np.array(Y_train)

Y_test = []
for y in y_test:
    value = np.abs(769-y)
    Y_test.append(value)
Y_test = np.array(Y_test)




In [59]:
print(y_train_valid.shape)
print(Y_train.shape)
print(Y_test.shape)
print(Y_test.shape)

(2115,)
(2115,)
(443,)
(443,)


In [60]:
# create feature and targets tensor for train set.  we create tensor, then we will create variable
featuresTrain = torch.from_numpy(X_train_valid)
targetsTrain = torch.from_numpy(Y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(X_test)
targetsTest = torch.from_numpy(Y_test).type(torch.LongTensor) # data type is long

In [61]:
# batch_size, epoch and iteration
batch_size = 100
n_iters = 10000
num_epochs = n_iters / (len(X_train_valid) / batch_size)
num_epochs = int(num_epochs)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)


In [62]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, 
                          nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out


In [ ]:

# Create RNN
dtype = torch.FloatTensor
input_dim = 1000    # input dimension
hidden_dim = 200  # hidden layer dimension
layer_dim = 6     # number of hidden layers
output_dim = 10   # output dimension

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

# Cross Entropy Loss 
error = nn.CrossEntropyLoss().type(dtype)


# SGD Optimizer
learning_rate = 0.05
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
seq_dim = 25  
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        train  = Variable(signals.view(-1, seq_dim, input_dim))
        labels = Variable(labels )
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(train.float())
        
        # Calculate softmax and ross entropy loss
        loss = error(outputs, labels)
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        if count % 10 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for signals, labels in test_loader:
                signals = Variable(signals.view(-1, seq_dim, input_dim))
                
                # Forward propagation
                outputs = model(signals.float())
                
                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 1)[1]
                
                # Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 1 == 0:
                # Print Loss
                print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data, accuracy))

Iteration: 10  Loss: 1.5840939283370972  Accuracy: 30 %
Iteration: 20  Loss: 1.3395147323608398  Accuracy: 39 %
Iteration: 30  Loss: 1.3117599487304688  Accuracy: 52 %
Iteration: 40  Loss: 1.456087589263916  Accuracy: 58 %
Iteration: 50  Loss: 1.2119991779327393  Accuracy: 59 %
Iteration: 60  Loss: 0.45253363251686096  Accuracy: 61 %
Iteration: 70  Loss: 0.8311282992362976  Accuracy: 59 %
Iteration: 80  Loss: 0.40881621837615967  Accuracy: 62 %
Iteration: 90  Loss: 0.9112553596496582  Accuracy: 58 %
Iteration: 100  Loss: 0.4276775121688843  Accuracy: 58 %
Iteration: 110  Loss: 0.0912146270275116  Accuracy: 53 %
Iteration: 120  Loss: 0.5784710049629211  Accuracy: 55 %
